In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from brisc.manuscript_analysis import barcodes_in_cells as bc_cells
from brisc.manuscript_analysis import match_to_library as match_lib
from brisc.manuscript_analysis import sensitivity as sens
from brisc.manuscript_analysis import mcherry_intensity as mcherry_int
from brisc.manuscript_analysis import distance_between_cells as dist_cells
from brisc.manuscript_analysis import spatial_plots_rabies as spatial

from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

matplotlib.rcParams["pdf.fonttype"] = 42  # for pdfs

import flexiznam as flz

In [ ]:
# Placeholder
# TODO: setup flexilims project
project = "barcode_diversity_analysis"
flexilims_session = flz.get_flexilims_session(project)
READ_ROOT = (
    flz.get_data_root("processed", flexilims_session=flexilims_session)
    / "rabies_manuscript_figures"
)
SAVE_ROOT = (
    flz.get_data_root("processed", flexilims_session=flexilims_session)
    / "rabies_manuscript_figures"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
reload = False

In [ ]:
(
    in_situ_perfect_match,
    random_perfect_match,
    rv35_library,
) = match_lib.load_data()

(
    unfiltered_bar_per_presynaptic_cell,
    unfiltered_bar_per_starter_cell,
    counts_with_starter,
    counts_without_starter,
    starters_per_barcode,
) = bc_cells.load_data()

good_cells, labeled_images, rabies_stack, spots_df = sens.load_data()

valid_mcherry = mcherry_int.load_mcherry_data()

(
    distances_same_bc,
    distances_diff_bc,
    nearest_diff_bc,
    adjacency_df,
    distances_same_bc_nonadj,
    distances_diff_bc_nonadj,
) = dist_cells.pairwise_barcode_distances_with_nearest_diff(
    data_path="becalia_rabies_barseq/BRAC8498.3e/chamber_07/"
)

cells = pd.read_pickle(
    "/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8498.3e/analysis/cell_barcode_df.pkl"
)

double_bc_data = bc_cells.load_double_barcode_data(
    data_path="/nemo/project/proj-znamenp-barseq/processed/becalia_rabies_barseq/BRAC8498.3e/analysis/"
)

In [ ]:
# Plot Fig.1
fontsize_dict = {"title": 7, "label": 7, "tick": 5, "legend": 5}
pad_dict = {"label": 1, "tick": 1, "legend": 5}
line_width = 0.9
line_alpha = 1

cm = 1 / 2.54
fig = plt.figure(figsize=(18 * cm, 18 * cm), dpi=350)
data_path = Path(
    "/nemo/lab/znamenskiyp/home/shared/projects/barcode_diversity_analysis/collapsed_barcodes/"
)
save_path = Path(
    "/nemo/lab/znamenskiyp/home/shared/projects/barcode_diversity_analysis/figures/"
)
save_fig = False
figname = "fig1_plasmid_barcoding_schema_library"

# 1 BC per cell, starter, presyn 5.11B,E X
# 2 Histograms matching in situ to library sequences X
# 3 Presyn cells per barcode X
# 4 Sensitivity of barcode detection (zoomed out image) X
# 5 Sensitivity of barcode detection histogram X
# 6 Number of presyns vs mCherry fluor X

# 7 Num starter per barcode histogram X
# 8 Distance between starters sharing barcodes histogram X
# 9 Spatial structure of cell locations (5.15C) X
# 10 Spatial structure of cell locations (5.15F) X
# 11 Stacked barchart of number of presyns with 1 or other barcode for double barcoded starter X
# 12 ML/AP plot of presyn distance 5.17 for shuffled/non shuffled and histo of distances

# Coord format: [left, bottom, width, height]
ax_locations = [
    [0.05, 0.67, 0.25, 0.25],  # Row 1, Col 1
    [0.36, 0.67, 0.25, 0.25],  # Row 1, Col 2
    [0.67, 0.67, 0.25, 0.25],  # Row 1, Col 3
    [0.05, 0.36, 0.25, 0.25],  # Row 2, Col 1
    [0.36, 0.36, 0.25, 0.25],  # Row 2, Col 2
    [0.67, 0.36, 0.25, 0.25],  # Row 2, Col 3
    [0.05, 0.05, 0.25, 0.25],  # Row 3, Col 1
    [0.36, 0.05, 0.25, 0.25],  # Row 3, Col 2
    [0.67, 0.05, 0.25, 0.25],  # Row 3, Col 3
    [0.05, -0.26, 0.85, 0.25],  # Row 4, Col 1
    [0.05, -0.57, 0.25, 0.25],  # Row 5, Col 1
    [0.36, -0.57, 0.25, 0.25],  # Row 5, Col 2
    [0.67, -0.57, 0.25, 0.25],  # Row 5, Col 3
    [1.03, -0.57, 0.125, 0.25],  # Row 5, Col 4
]

# Add blank squares with text
ax_bc_per_cell_presyn = fig.add_axes(ax_locations[0])
im = bc_cells.plot_bc_per_cell_presyn(
    unfiltered_bar_per_presynaptic_cell,
    ax=ax_bc_per_cell_presyn,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

ax_bc_per_cell_starter = fig.add_axes(ax_locations[1])
im = bc_cells.plot_bc_per_cell_starter(
    unfiltered_bar_per_starter_cell,
    ax=ax_bc_per_cell_starter,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    padding=28,
)

ax_matches_to_lib = fig.add_axes(ax_locations[2])
im = match_lib.plot_matches_to_library(
    in_situ_perfect_match,
    random_perfect_match,
    rv35_library,
    ax=ax_matches_to_lib,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
)

ax_presyn_orphans = fig.add_axes(ax_locations[3])
im = bc_cells.plot_presyn_per_orphan(
    counts_with_starter,
    counts_without_starter,
    ax=ax_presyn_orphans,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
)

ax_sensitivity_histo = fig.add_axes(ax_locations[4])
im = sens.plot_histogram_bc_per_cell(
    good_cells,
    ax=ax_sensitivity_histo,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

ax_spots_in_cells = fig.add_axes(ax_locations[5])
im = sens.plot_cells_spots(
    good_cells,
    rabies_stack,
    labeled_images,
    ax=ax_spots_in_cells,
    roi_of_interest=5,
    min_y=450,
    max_y=1100,
    min_x=400,
    max_x=1000,
    colors=[(1, 0, 0), (0, 1, 1)],
    vmaxs=[800, 200],
    linewidth=0.9,
)

ax_mcherry_fluor = fig.add_axes(ax_locations[6])
im = mcherry_int.plot_mcherry_intensity_presyn(
    valid_mcherry,
    ax=ax_mcherry_fluor,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    spot_size=5,
)

ax_spots_in_cells = fig.add_axes(ax_locations[7])
im = bc_cells.plot_starters_per_barcode(
    starters_per_barcode,
    ax=ax_spots_in_cells,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    padding=100,
)

ax_dist_between_starters = fig.add_axes(ax_locations[8])
im = dist_cells.plot_dist_between_starters(
    distances_diff_bc_nonadj,
    distances_same_bc_nonadj,
    ax=ax_dist_between_starters,
    bins=40,
    max_dist=2,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    line_width=line_width,
)

ax_dist_between_starters = fig.add_axes(ax_locations[9])
spatial.plot_flat_ml_rv_cells(
    cells,
    ax=ax_dist_between_starters,
    label_fontsize=7,
    tick_fontsize=5,
    presyn_size=1,
    starter_size=2,
    presyn_alpha=0.7,
    legend_height=1.8,
)

ax_v1_rabies = fig.add_axes(ax_locations[10])
ax_density = fig.add_axes(ax_locations[11])
spatial.plot_rabies_cells(
    ax_v1_rabies,
    ax_density,
    cells,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
)

ax_stack = fig.add_axes(ax_locations[12])
ax_violin = fig.add_axes(ax_locations[13])
bc_cells.plot_double_barcode_barstack(
    double_bc_data,
    ax_stack,
    ax_violin,
    label_fontsize=fontsize_dict["label"],
    tick_fontsize=fontsize_dict["tick"],
    barcode_proportion=True,
    swarmplot_dotsize=2,
)


if save_fig:
    pdf_path = f"{save_path}/figures/{figname}.pdf"
    png_path = f"{save_path}/figures/{figname}.png"
    fig.savefig(pdf_path, format="pdf", transparent=True, bbox_inches="tight")
    fig.savefig(png_path, format="png", transparent=True, bbox_inches="tight")
    print(f"Saved PDF figure to {pdf_path}")
    print(f"Saved PNG figure to {png_path}")